# Q&A over the Code Base to Understand How it Works

- Anything the person is not understanding about the project, he is gonna use that
- user will upload one github repro url
- from url , clone the repro
- Extract the information
- rag operation

In [2]:
from git import Repo # Clone any repro from github
from langchain.text_splitter import Language # With the help of this i will set python programming
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding Model
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationSummaryMemory # Creating memory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'e:\\Source-Code-Analysis-Generative-AI\\research'

In [4]:
# Make Directory
!mkdir test_repro


A subdirectory or file test_repro already exists.


In [6]:
# Clone the repro in test_repro
repo_path="test_repro/"
repo=Repo.clone_from("https://github.com/PriyanshuDey23/Medical-ChatBot-using-GenAi", to_path=repo_path)

In [7]:
# Load all the codes from the repository(test_repro)
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"], # Python
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [8]:
documents = loader.load()

In [9]:
# Load every code from python in document format
documents

[Document(metadata={'source': 'test_repro\\ test.py', 'language': <Language.PYTHON: 'python'>}, page_content=''),
 Document(metadata={'source': 'test_repro\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask , render_template ,jsonify ,request\nfrom src.helper import google_embeddings\nfrom pinecone import Pinecone, ServerlessSpec\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding Model\nfrom dotenv import load_dotenv\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom src.prompt import *\nimport os\n\n\napp = Flask(__name__)\n\n# Set Environment variable\n\nload_dotenv()\n# Load the api\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\n\n# Load The Google Api key\nGOOGLE

In [10]:
len(documents)

8

In [11]:
documents[1]


Document(metadata={'source': 'test_repro\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask , render_template ,jsonify ,request\nfrom src.helper import google_embeddings\nfrom pinecone import Pinecone, ServerlessSpec\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding Model\nfrom dotenv import load_dotenv\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom src.prompt import *\nimport os\n\n\napp = Flask(__name__)\n\n# Set Environment variable\n\nload_dotenv()\n# Load the api\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\n\n# Load The Google Api key\nGOOGLE_API_KEY = os.environ.get(\'GOOGLE_API_KEY\')\n\n# Embedding\nembeddings=google_embeddings()\n\n# Load Existing ind

In [12]:
# Context Aware Splitting
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [13]:
texts = documents_splitter.split_documents(documents)

In [14]:
texts

[Document(metadata={'source': 'test_repro\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask , render_template ,jsonify ,request\nfrom src.helper import google_embeddings\nfrom pinecone import Pinecone, ServerlessSpec\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding Model\nfrom dotenv import load_dotenv\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain'),
 Document(metadata={'source': 'test_repro\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from langchain_core.prompts import ChatPromptTemplate\nfrom src.prompt import *\nimport os'),
 Document(metadata={'source': 'test_repro\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='app = Flask(__name__)\n\n# Set Environment variable\n\nload_dotenv()\n# Lo

In [15]:
len(texts)

18

In [16]:
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [17]:
# Embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [18]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')
# Db is created in research

In [19]:
vectordb.persist()

C:\Users\HP\AppData\Local\Temp\ipykernel_3636\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [20]:
# LLm
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002",temperature=0.3, max_tokens=500)

In [22]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [23]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [26]:
question = "what is create_index funtion?"

In [27]:
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


The `create_index` function creates a Pinecone index named "medicalbot" with a dimension of 768, using the cosine similarity metric.  It is configured as a serverless index on AWS in the us-east-1 region.  It does not appear to be explicitly defined as a Python function, but rather a method call on the Pinecone client object.



In [28]:
question = "what is load_pdf_file funtion?"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


The `load_pdf_file` function loads all PDF files from a specified directory. It uses the `DirectoryLoader` with `PyPDFLoader` to achieve this, returning a list of `Document` objects.

